In [1]:
%load_ext autoreload
%autoreload 2

# UVIS G280 Simulator compared to published data
This notebook runs the simulator and compares the output to the data published in Wakeford et al. (2020, AJ)
We then plot the results to visually show the application of the simulator. 



In [2]:
import UVIS_precision_calculator as upc
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt 
import matplotlib.gridspec as grid
import seaborn as sns

sns.set(style="ticks", palette="colorblind", color_codes=True, context="talk")
FIG_WIDE = (11, 8)

In [3]:
%matplotlib qt5

Run the UVIS simulator on the file you want. We will follow the example in the UVIS precision calculator

#### INPUT

`data_folder = '..Data/'`
	The place where the ETC output file located
	The ETC file needs to contain the following information in the correct structure. 
	This file can be self generated if also in this structure.
	(wavelength (A), readnoise, dark_counts, sky_counts, target_counts, total_counts)
	Columns 0 and 4 are used in this calculation, please maintain these column 
	positions for self input files
	Wavelength is input from this file in Angstroms 

`file_name = ['H41_ETC']`
	This can be an array of filenames 
	make sure it is always in square brackets

`exposure_time = [190]` 
	This is an array of exposuretimes corresponding to the 
	filenames listed in file_name
	make sure it is always in square brackets

`orbits_in_transit = [2]` 
	This is an array of the number of HST orbits inside a single
	transit (not the total observation) corresponding to the
	filenames listed in file_name
	make sure it is always in square brackets

`no_of_transits = [2]` 
	This is an array of the number of transits of each targets 
	in your program corresponding to the filenames listed in
	file_name make sure it is always in square brackets

`wl = [2000,8000]`
	default value spans the maximum range of the grism. This will 
	calculate the broadband signal that will be the first line of
	the produced file. 

`startw = 2000`
	This is the starting range for your spectral bins.
	Default is at minimum = 2000 angstroms. 
	Where this might be changed is for small stars with little 
	flux in the short wavelengths. Or if simulating the -1 order

`endw = 8000`
	This is the end of the spectral binning range
	Default set to the maximum, 8000 angstroms

`binlen = 100`
	This is in angstroms and specifies the size of the 
	spectral bins in wavlength space. 
	Minimum recommended = 60 angstorms (Wakeford et al 2020 = 100)

`out_name = 'string'`
	This is what you want the output specified as in addition 
	to the input name.

#### OUTPUT

data file for each input planet file:
output name = {data_folder}{file_name}.UVIS_{out_name}_sim.txt'
	contains the broadband {wl} and spectroscopic precisions ({binlen} bins) 
	wavelength (nm), bin size (nm), precision per transit (ppm), precision per target (ppm)


In [20]:
data_folder = ""

file_name = ["H41_ETC", "H23_ETC"] 

exposure_time = [190, 190] 

orbits_in_transit = [2, 1] # floor(Transit duration / HST period ≈ 96 minutes)

no_of_transits = [2, 6] 

wl = [2000,8000]

startw = 2000

endw = 8000

binlen = 100

out_name = "10nm_transmission"

upc.UVIS_simulation(data_folder,file_name,exposure_time,orbits_in_transit,no_of_transits,wl,startw,endw,binlen,out_name)

H41_ETC
exp time =  190
exp in transit =  23.0
exp per orbit =  12.0
precision/transit =  37.94149373653571
(60, 4)
0 [2000, 8000] 100 ['H41_ETC', 'H23_ETC'] [190 190] [2 1] [2 6]
Data has been saved as:  H41_ETC.UVIS_10nm_transmission_sim.csv

H23_ETC
exp time =  190
exp in transit =  12.0
exp per orbit =  12.0
precision/transit =  70.51289074776273
(60, 4)
1 [2000, 8000] 100 ['H41_ETC', 'H23_ETC'] [190 190] [2 1] [2 6]
Data has been saved as:  H23_ETC.UVIS_10nm_transmission_sim.csv



In the next cell we will plot the two datasets from Wakeford et al. (2020) against the simulated data from this calculator. 

In [19]:
######
# Load
######
h23_sim = upc.read_UVIS("H23_ETC.UVIS_10nm_transmission_sim.csv")
h41_sim = upc.read_UVIS("H41_ETC_default.UVIS_10nm_transmission_sim.csv")
h41_data = np.loadtxt('H41_data_Wakeford2020.txt')

######
# Plot
######
fig, ax = plt.subplots(figsize=FIG_WIDE)

upc.plot_prec(ax, h23_sim, label='HP23 ETC')
upc.plot_prec(ax, h41_sim, label='HP41 ETC')
upc.plot_prec_wakeford(ax, h41_data)
ax.hlines(33, 100, 1000, color='k', linestyle='--')

ax.set_xlim(180,820)
ax.set_ylim(0,1000)

ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel('Precision (ppm)')
ax.legend(fontsize=10)

plt.tight_layout()

This shows that the simulator is a good approximation of the results presented in Wakeford et al. (2020) with some minor differences at the short and long wavelengths. Overall we expect this simulator to be used for proposal purposes only and is an approximation of the expected precision. 